# LIBRARY

In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

from sklearn.linear_model import LinearRegression
from statsmodels.formula.api import ols
from statsmodels.formula.api import glm
from statsmodels.genmod.families.family import Binomial
from sklearn.ensemble import RandomForestClassifier

In [2]:
import warnings
warnings.filterwarnings('ignore')

# DATA

In [3]:
train = pd.read_csv('../Data/train_preprocessing_cluster.csv')
val = pd.read_csv('../Data/valid_preprocessing_cluster.csv')
test = pd.read_csv('../Data/test_preprocessing_cluster.csv')
sub = pd.read_csv('../Data/PJT002_submission.csv')

### 필요한 열만 가져오기

In [4]:
# columns = ['id', 'dt_of_fr', 'bldng_cnt', 'bldng_ar', 'ttl_ar', 'lnd_ar'
#            , 'jmk', 'fr_sttn_dstnc', 'fr_wthr_fclt_dstnc'
#            , 'mlt_us_yn', 'cctv_dstnc', 'cctv_in_100m'
#            , 'fr_wthr_fclt_in_100m', 'tbc_rtl_str_dstnc', 'sft_emrgnc_bll_dstnc'
#            , 'ahsm_dstnc', 'no_tbc_zn_dstnc', 'bldng_cnt_in_50m', 'fr_yn'
#           ]
# columns2 = ['id', 'dt_of_fr', 'bldng_cnt', 'bldng_ar', 'ttl_ar', 'lnd_ar'
#            , 'jmk', 'fr_sttn_dstnc', 'fr_wthr_fclt_dstnc'
#            , 'mlt_us_yn', 'cctv_dstnc', 'cctv_in_100m'
#            , 'fr_wthr_fclt_in_100m', 'tbc_rtl_str_dstnc', 'sft_emrgnc_bll_dstnc'
#            , 'ahsm_dstnc', 'no_tbc_zn_dstnc', 'bldng_cnt_in_50m'
#           ]

# train_pp = train[columns]
# test_pp = test[columns2]
# val_pp = val[columns]

# NULL CHECK
null 없는 애들로만 가져와서 null이 없음

In [4]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59191 entries, 0 to 59190
Data columns (total 33 columns):
id                      59191 non-null int64
tag                     59191 non-null object
year                    59191 non-null int64
month                   59191 non-null int64
day                     59191 non-null int64
dayofweek               59191 non-null object
year-month              59191 non-null object
year-month-day          59191 non-null object
hour                    59191 non-null int64
minute                  59191 non-null int64
second                  59191 non-null int64
jmk                     59191 non-null object
mlt_us_yn(encode)       59191 non-null bool
tmprtr                  59191 non-null float64
wnd_spd                 59191 non-null float64
wnd_drctn               59191 non-null float64
hmdt                    59191 non-null float64
address                 59191 non-null object
address1                59191 non-null object
address2              

In [8]:
train = train[train['address1'] != '창원시']

In [ ]:
train['address1'] != '창원시'

# MAKING DATASET

In [105]:
# choose independent variables(Xs) which are useful!
# 트레이닝에 사용할 변수 목록을 적어주세요.
independents = [
#     'bldng_cnt', 'bldng_ar', 'ttl_ar', 'lnd_ar', 
       'jmk', 
    'address1',
    'fr_wthr_fclt_in_100m', 'mlt_us_yn(encode)',
       'tbc_rtl_str_dstnc',
    
#     'fr_sttn_dstnc', 'fr_wthr_fclt_dstnc',  'cctv_dstnc',
#        'cctv_in_100m', 
#        'sft_emrgnc_bll_dstnc', 'ahsm_dstnc', 'no_tbc_zn_dstnc',
#        'bldng_cnt_in_50m',  
    'month',
#         'year-month'
] 
# 독립변수
dependent = ['fire_yes'] # 종속변수

In [106]:
# 적어준 변수 목록을 사용해 데이터를 트레이닝에 맞는 포맷으로 변경합니다.
train_X = train[independents]
train_y = train[dependent]

test_X = test[independents]
val_X = val[independents]

# MODEL FITTING

## 선형 회귀

In [17]:
reg = LinearRegression().fit(train_X, train_y)
# 대문자는 매트릭스, 소문자는 벡터

In [18]:
# x 계수
# x 계수의 의미를 해석하세요.
reg.coef_
# 피클래스가 한 계단 올라갈 때마다 죽음에 0.19배 가까워짐

array([[ 2.23635929e-02,  4.00456635e-01, -1.01209357e-05,
        -2.37880224e-03]])

In [19]:
# 절편
reg.intercept_

array([0.19475158])

In [20]:
reg.score(train_X, train_y, sample_weight=None)

0.01936329292503114

## 일반화 선형모형

In [ ]:
res = glm('fire_yes ~ fr_wthr_fclt_in_100m + tbc_rtl_str_dstnc + month', train, family=Binomial()).fit()
res.summary()

In [107]:
res = glm('fire_yes ~ C(address1)*C(jmk)', train, family=Binomial()).fit()
res.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Generalized Linear Model Regression Results                           
=================================================================================================
Dep. Variable:     ['fire_yes[False]', 'fire_yes[True]']   No. Observations:                50749
Model:                                               GLM   Df Residuals:                    50382
Model Family:                                   Binomial   Df Model:                          366
Link Function:                                     logit   Scale:                          1.0000
Method:                                             IRLS   Log-Likelihood:                    nan
Date:                                   Fri, 01 Nov 2019   Deviance:                          nan
Time:                                           13:05:35   Pearson chi2:                 5.73e+19
No. Iterations:                                      100   Covariance Type:             nonrobust
==================================================================================================
                                     coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------------------
Intercept                       4.504e+15   3.87e+07   1.16e+08      0.000     4.5e+15     4.5e+15
C(address1)[T.거창군]             -5.338e+15   7.75e+07  -6.89e+07      0.000   -5.34e+15   -5.34e+15
C(address1)[T.고성군]              8.652e+16   7.75e+07   1.12e+09      0.000    8.65e+16    8.65e+16
C(address1)[T.남해군]              5.445e+15   9.76e+06   5.58e+08      0.000    5.44e+15    5.44e+15
C(address1)[T.밀양시]              1.855e+17   7.75e+07   2.39e+09      0.000    1.85e+17    1.85e+17
C(address1)[T.사천시]             -1.656e+15   5.13e+07  -3.23e+07      0.000   -1.66e+15   -1.66e+15
C(address1)[T.산청군]             -9.121e+14   7.75e+07  -1.18e+07      0.000   -9.12e+14   -9.12e+14
C(address1)[T.양산시]             -3.157e+15   6.13e+07  -5.15e+07      0.000   -3.16e+15   -3.16e+15
C(address1)[T.의령군]              1.266e+16   1.03e+07   1.23e+09      0.000    1.27e+16    1.27e+16
C(address1)[T.진주시]             -1.617e+15   5.48e+07  -2.95e+07      0.000   -1.62e+15   -1.62e+15
C(address1)[T.창녕군]             -7.948e+15   7.75e+07  -1.03e+08      0.000   -7.95e+15   -7.95e+15
C(address1)[T.통영시]              1.825e+15   5.48e+07   3.33e+07      0.000    1.83e+15    1.83e+15
C(address1)[T.하동군]             -2.632e+15   8.93e+06  -2.95e+08      0.000   -2.63e+15   -2.63e+15
C(address1)[T.함안군]             -5.982e+14   8.63e+06  -6.93e+07      0.000   -5.98e+14   -5.98e+14
C(address1)[T.함양군]              3.937e+16    9.9e+06   3.98e+09      0.000    3.94e+16    3.94e+16
C(address1)[T.합천군]              5.536e+15   9.84e+06   5.63e+08      0.000    5.54e+15    5.54e+15
C(jmk)[T.과]                     3.373e+16   6.13e+07   5.51e+08      0.000    3.37e+16    3.37e+16
C(jmk)[T.구]                    -3.684e+15   4.33e+07  -8.51e+07      0.000   -3.68e+15   -3.68e+15
C(jmk)[T.답]                    -3.824e+15   3.92e+07  -9.76e+07      0.000   -3.82e+15   -3.82e+15
C(jmk)[T.대]                    -3.439e+15   3.88e+07  -8.87e+07      0.000   -3.44e+15   -3.44e+15
C(jmk)[T.도]                    -9.043e+13   4.08e+07  -2.21e+06      0.000   -9.04e+13   -9.04e+13
C(jmk)[T.목]                    -6.206e+15    4.3e+07  -1.44e+08      0.000   -6.21e+15   -6.21e+15
C(jmk)[T.묘]                    -9.397e+14   7.75e+07  -1.21e+07      0.000    -9.4e+14    -9.4e+14
C(jmk)[T.사]                    -3.368e+16    2.7e+07  -1.25e+09      0.000   -3.37e+16   -3.37e+16
C(jmk)[T.수]                    -1.645e+15   6.13e+07  -2.69e+07      0.000   -1.65e+15   -1.65e+15
C(jmk)[T.양]                     6.214e+15   4.18e+07   1.48e+08      0.000    6.21e+15    6.21e+15
C(jmk)[T.원]                     -2.77e+15    4.9e+07  -5.65e+07      0.000   -2.77e+15   -2.77e+15
C(

## 최소제곱추정 선형회귀

In [87]:
res = ols('fire_yes ~ month', train).fit()
# res.summary()

In [88]:
res.summary()

ValueError: shapes (50749,2) and (50749,2) not aligned: 2 (dim 1) != 50749 (dim 0)

## 값 예측하기

In [108]:
prediction = res.predict(test_X)

PatsyError: Error converting data to categorical: observation with value '김해시' does not match any of the expected levels (expected: ['거제시', '거창군', ..., '함양군', '합천군'])
    fire_yes ~ C(address1)*C(jmk)
               ^^^^^^^^^^^

In [109]:
sorted(prediction, reverse=True)

[0.9999999999417379,
 0.9917355371900816,
 0.9917355371900816,
 0.9917355371900816,
 0.9917355371900816,
 0.9917355371900816,
 0.9917355371900816,
 0.9917355371900816,
 0.9917355371900816,
 0.9917355371900816,
 0.9917355371900816,
 0.9917355371900816,
 0.9917355371900816,
 0.9917355371900816,
 0.9917355371900816,
 0.9899999999999993,
 0.9899999999999993,
 0.9899999999999993,
 0.9899999999999993,
 0.9899999999999993,
 0.9899999999999993,
 0.9791666666666653,
 0.9791666666666653,
 0.9791666666666653,
 0.9791666666666653,
 0.9791666666666653,
 0.9695885509838962,
 0.9695885509838962,
 0.9695885509838962,
 0.9695885509838962,
 0.9695885509838962,
 0.9695885509838962,
 0.9695885509838962,
 0.9695885509838962,
 0.9695885509838962,
 0.9695885509838962,
 0.9695885509838962,
 0.9695885509838962,
 0.9695885509838962,
 0.9677419354838709,
 0.9677419354838709,
 0.9677419354838709,
 0.9451476793248954,
 0.9451476793248954,
 0.9451476793248954,
 0.9451476793248954,
 0.9451476793248954,
 0.9451476793

In [98]:
prediction_binomial = [1 if p>=0.3 else 0 for p in prediction]

# VALIDATION 파일 이용하기

In [41]:
prediction_val = res.predict(val_X)
sorted(prediction_val, reverse=True)

[0.9068171198939408,
 0.9064486786279641,
 0.9051466644158523,
 0.8996389810845806,
 0.896407473646308,
 0.8952653999191234,
 0.8951169006193903,
 0.8949053019823713,
 0.8947468614743204,
 0.8947097671189543,
 0.8934003818588635,
 0.8933203662912034,
 0.8921100954134039,
 0.8914967894360755,
 0.8910174269828216,
 0.8900312957794704,
 0.8897937712551934,
 0.8891836473146604,
 0.8890786825164142,
 0.889010376176495,
 0.8887785778932198,
 0.8881560319657672,
 0.8881093944443159,
 0.8878558843028926,
 0.8874638196242107,
 0.887003698104928,
 0.887003698104928,
 0.8869687123121752,
 0.8864049102185289,
 0.8855177684424669,
 0.8849148833068851,
 0.8848542347264456,
 0.8847005854454982,
 0.8844532317759597,
 0.8843671794060972,
 0.8843445593217321,
 0.8839772421638609,
 0.8838072440143023,
 0.8837181847542089,
 0.883634056513132,
 0.8836087207269014,
 0.8835478570339992,
 0.8834827572190563,
 0.8832519826404156,
 0.8828740314161991,
 0.8828539560338912,
 0.8824183525777645,
 0.882418352577764

In [103]:
prediction_binomial_val = [1 if p>=0.9 else 0 for p in prediction_val]

# RANDOM FOREST

In [43]:
forest = RandomForestClassifier(random_state=42, n_estimators=10)

In [44]:
forest = forest.fit(train_X, train_y)

In [45]:
forest.feature_importances_

array([0.01556735, 0.00502622, 0.94084573, 0.03856071])

In [46]:
predict_forest = forest.predict(val_X)

In [47]:
predict_forest

array([False, False, False, ..., False, False, False])

In [65]:
# MSE
def mse(prediction, validation):
    return np.mean((prediction - validation) ** 2) 

In [104]:
mse(prediction_binomial_val, val['fire_yes'])

0.18454624528848942

In [69]:
mse([0] * len(val['fire_yes']), val['fire_yes'])

0.18440127573209625

# F1 구하기 Precision/Recall

In [48]:
import sklearn.metrics as metrics

In [52]:
print('f1', metrics.f1_score(prediction_binomial_val,val['fire_yes']) )

f1 0.3113831089351285


In [54]:
print('f1', metrics.f1_score(predict_forest,val['fire_yes']) )

f1 0.17750439367311072


# 한 번에 모델 돌리기

In [55]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB

In [56]:
def train_and_test(model):
    a = model.fit(train_X, train_y)
    prediction = a.predict(test_X)
    accuracy = round(model.score(train_X, train_y) * 100, 2)
    print("Accuracy : ", accuracy, "%")
    return prediction

In [33]:
log_pred = train_and_test(LogisticRegression())

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Accuracy :  87.5 %


In [34]:
# svm_pred = train_and_test(SVC())

In [35]:
# knn_pred_4 = train_and_test(KNeighborsClassifier(n_neighbors = 4))

In [36]:
rf_pred = train_and_test(RandomForestClassifier(n_estimators=100))

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


Accuracy :  99.94 %


In [37]:
# nb_pred = train_and_test(GaussianNB())

# VALIDATION 한 번에 돌리기

In [85]:
def train_and_val(model):
    a = model.fit(train_X, train_y)
    prediction_val = a.predict(val_X)
    prediction_binomial_val = [1 if p>=0.3 else 0 for p in prediction_val]
    print('f1:', metrics.f1_score(prediction_binomial_val,val_pp['fr_yn']) )
    return prediction_val

In [86]:
train_and_val(LogisticRegression())

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


f1: 0.20277777777777778


array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [87]:
train_and_val(RandomForestClassifier(n_estimators=100))

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


f1: 0.383634431455898


array([1, 0, 0, ..., 0, 0, 0], dtype=int64)

In [88]:
train_and_val(KNeighborsClassifier(n_neighbors = 4))

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  


f1: 0.31120552310143385


array([0, 0, 0, ..., 0, 0, 1], dtype=int64)

In [89]:
train_and_val(GaussianNB())

f1: 0.20207612456747404


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [ ]:
train_and_val(SVC())

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
